# Day 36 - Stock Trading News Alert

### Instructions

In this project, you will create a Python application that will notify you when a stock has reached a certain price and print relevant news articles.


STEP 1: Use https://www.alphavantage.co
When STOCK price increase/decreases by 5% between yesterday and the day before yesterday then print("Get News").

STEP 2: Use https://newsapi.org
Instead of printing ("Get News"), actually get the first 3 news pieces for the COMPANY_NAME. 

STEP 3: Use https://www.twilio.com
Send a seperate message with the percentage change and each article's title and description to your phone number. 

### How to use this program:

1. Change the STOCK and COMPANY_NAME to the stock you want to track.
2. Run all cells
3. Look at the output on the bottom of the page.

In [10]:
import requests
import os

In [19]:
stockmarket_api_key = os.getenv("STOCK_MARKET_API")  # https://www.alphavantage.co/
news_api_key = os.getenv("NEWS_API_KEY")  # https://newsapi.org/


# Stocks List: https://www.slickcharts.com/sp500
STOCK = "MSFT"
COMPANY_NAME = "Microsoft Corp."

### Fetch StockMarket Data from Alpha Vantage API 

Alpha Vantage API is used to fetch the stock market data. The API is free to use and does not require any authentication. The API provides the data in JSON format. The API provides the data for the following fields: Open, High, Low, Close, Volume. The API provides the data for the following stock market indices: NSE, BSE, NASDAQ, NYSE, S&P 500, Dow Jones, FTSE 100, DAX, CAC 40, Nikkei 225, Hang Seng.

Alpha Vantage API Documentation: https://www.alphavantage.co/documentation/

In [12]:
params = {
    "function": "TIME_SERIES_DAILY",
    "symbol": STOCK,
    "apikey": stockmarket_api_key,
}

url = "https://www.alphavantage.co/query"
r = requests.get(url, params=params)
print(r.json())

time_series_daily = r.json()["Time Series (Daily)"]

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'MSFT', '3. Last Refreshed': '2023-11-08', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2023-11-08': {'1. open': '361.6800', '2. high': '363.8700', '3. low': '360.5500', '4. close': '363.2000', '5. volume': '26767828'}, '2023-11-07': {'1. open': '359.4000', '2. high': '362.4600', '3. low': '357.6300', '4. close': '360.5300', '5. volume': '25833931'}, '2023-11-06': {'1. open': '353.4500', '2. high': '357.5400', '3. low': '353.3500', '4. close': '356.5300', '5. volume': '23828301'}, '2023-11-03': {'1. open': '349.6300', '2. high': '354.3900', '3. low': '347.3300', '4. close': '352.8000', '5. volume': '23637673'}, '2023-11-02': {'1. open': '347.2400', '2. high': '348.8300', '3. low': '344.7700', '4. close': '348.3200', '5. volume': '24348072'}, '2023-11-01': {'1. open': '339.7900', '2. high': '347.4200', '3. low': '339.6500', '4. close': '346.0700', '5

Focus on the yesterday's and the day before yesterday's opening price of the ``STOCK``. Other Options are:
- ``'1. open'``
- ``'2. high'``
- ``'3. low'``
- ``'4. close'``
- ``'5. volume'``

In [13]:
closing_price_hist = [
    value["4. close"] for (_, value) in time_series_daily.items()
]

In [14]:
def detect_price_change(closing_price_hist):
    """ Detects if there is a 5% change in the closing price of the stock in the last two days.
        Returns True if there is a 5% change, else returns False."""
    yesterday_closing_price = float(closing_price_hist[0])
    day_before_yesterday_closing_price = float(closing_price_hist[1])
    difference = abs((yesterday_closing_price / day_before_yesterday_closing_price) - 1)
    
    return difference >= 0.05

### Fetch Market news & sentiments from Alphavantage API

This API returns live and historical market news & sentiment data from a large & growing selection of premier news outlets around the world, covering stocks, cryptocurrencies, forex, and a wide range of topics such as fiscal policy, mergers & acquisitions, IPOs, etc.

In [15]:
def fetch_news():
    """ Gets the first 3 news articles for the stock from https://www.alphavantage.co/queryo."""
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": STOCK,
        "apikey": stockmarket_api_key,
    }   

    url = "https://www.alphavantage.co/queryo"
    r = requests.get(url, params=params)
    news_data = r.json()
    return news_data["feed"][:3]

### Fetch Data from NewsAPI

NewsAPI is used to locate articles and breaking news headlines from news sources and blogs across the web with JSON API. The API is free to use and does not require any authentication. The API provides the data for the following fields: Source, Author, Title, Description, URL, URL to image, Published at, Content.

In [16]:
import datetime as dt

now = dt.datetime.now()

# Last 14 days
last_14_days = now - dt.timedelta(days=14)

params = {
    "apikey": news_api_key,
    "q": COMPANY_NAME,
    "from": last_14_days,
    "to": now,
    "language": "en",
    "sortBy": "relevancy", # "popularity" or "relevancy"
}

url = "https://newsapi.org/v2/everything"

r  = requests.get(url, params=params)
articles = r.json()["articles"]

In [17]:
def format_news(articles):
    format_str = "{title}\n{url}\n{description}\n-------------------------"
    formatted_articles = [format_str.format(**article) for article in articles[:3]]
    return "\n".join(formatted_articles)

### Conclusion

The News from NewsAPI seem to provide a better result than the news from Alpha Vantage API in terms of relevancy. So for the final project task we will be using the news from NewsAPI.

In [18]:
if detect_price_change(closing_price_hist):
    print(f"There is a 5% change in the closing price of {COMPANY_NAME} in the last two days. Fetching news articles...\n")
    print(format_news(articles))
else: 
    print(f"The Stock price of {COMPANY_NAME} has not changed by 5% in the last two days.")   

The Stock price of Microsoft Corp. has not changed by 5% in the last two days.


### Ideas for Improvement

- print {STOCK} 🔺/🔻 {rounded_percent_diff} based on the change in the stock price.
- send SMS / Email 